In [1]:
using TaylorSeries, TaylorIntegration
using Plots, LaTeXStrings
# plotly()
pyplot(grid = :false, legend = :false, size=(600,600), color=:black, markersize = 0.3 )
const Z = 2.0
const order = 20
const t0 = 0.0
const t_max = 0.5 
const abs_tol = 1.0E-20
const steps = 500_000

500000

In [2]:
function undrivenHelium!(τ, q, dq)
    Q₁, Q₂, P₁, P₂ = q
    
    t1 = Q₁^2
    t2 = Q₂^2
    t = t1 * t2
    R12 = t1 - t2
    aR12 = abs(R12)
    RRR = aR12^3
    c1 = R12/RRR
   
    dq[1] = 0.25*t2*P₁
    dq[3] = 2*Q₁*(-0.125*P₂^2 + Z  - t2 - t2/aR12 + t*c1)
    dq[2] = 0.25*t1*P₂
    dq[4] = 2*Q₂*(-0.125*P₁^2 + Z  - t1 - t1/aR12 - t*c1)

#    return [t, q₁, q₂, p₁, p₂] 
    nothing
end

var1 = set_variables("t q1 q2 p1 p2", order = 1)


function condini(x10::Float64, px10::Float64)
    @assert x10 != 0
    Q1 = sqrt(x10)
    Q2 = 0.0
    P1 = px10*sqrt(x10)    
    P2 = sqrt(8Z)    
    return Float64[Q1, Q2, P1, P2]
end

function regHam(t, Q₁, Q₂, P₁, P₂)
    #Cantidades auxiliares
    P₁² = P₁^2
    P₂² = P₂^2
    Q₁² = Q₁^2
    Q₂² = Q₂^2
    nf = abs(Q₁² - Q₂²)
    
    H = 0.125*(P₁²*Q₂² + P₂²*Q₁²) - Z*(Q₁² + Q₂²) + Q₁²*Q₂²*(1.0 + 1.0/nf)
    return H
end


regHam(v) = regHam(v...)

regHam (generic function with 2 methods)

In [3]:
function vareqs_taylorinteg{T<:Number}(f, q0::Array{T,1}, t0::T, tmax::T,
        order::Int, abstol::T; maxsteps::Int=500)
    # Allocation
    const tv = Array{T}(maxsteps+1)
    dof = length(q0)
    const xv = Array{T}(dof, maxsteps+1)
    const jt = eye(T, dof)
    const vT = zeros(T, order+1)
    vT[2] = one(T)

    # NOTE: This changes GLOBALLY internal parameters of TaylorN
    global _δv = set_variables("δ", order=1, numvars=dof)

    # Initial conditions
    @inbounds tv[1] = t0
    for ind in 1:dof
        @inbounds xv[ind,1] = q0[ind]
    end
    const x0 = vcat(q0, reshape(jt, dof*dof))
    nx0 = dof*(dof+1)
    t00 = t0

    # Initialize the vector of Taylor1 expansions
    const x = Array{Taylor1{T}}(nx0)
    for i in eachindex(x0)
        @inbounds x[i] = Taylor1( x0[i], order )
    end

    #Allocate auxiliary arrays
    const dx = Array{Taylor1{T}}(nx0)
    const xaux = Array{Taylor1{T}}(nx0)
    const δx = Array{TaylorN{Taylor1{T}}}(dof)
    const dδx = Array{TaylorN{Taylor1{T}}}(dof)
    const jac = Array{Taylor1{T}}(dof,dof)
    for i in eachindex(jac)
        @inbounds jac[i] = zero(x[1])
    end

    #auxiliary arrays for symplectic structure tests
    const δSv = Array{T}(maxsteps+1); δSv[1] = zero(T)
    auxJn = Int(dof/2)
    const J_n = vcat(  hcat(zeros(auxJn,auxJn), eye(auxJn,auxJn)), hcat(-eye(auxJn,auxJn), zeros(auxJn,auxJn))  )

    # Integration
    nsteps = 1
    while t0 < tmax
        δt = TaylorIntegration.liap_taylorstep!(f, x, dx, xaux, δx, dδx, jac, t0, tmax, x0, order, abstol, vT)
        for ind in eachindex(jt)
            @inbounds jt[ind] = x0[dof+ind]
        end
        t0 += δt
        tspan = t0-t00
        nsteps += 1
        @inbounds tv[nsteps] = t0
        @inbounds for ind in 1:dof
            xv[ind,nsteps] = x0[ind]
        end
        δSv[nsteps] = norm( jt'*J_n*jt-J_n, Inf)
        if nsteps > maxsteps
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
    end

    return view(tv,1:nsteps),  view(transpose(xv),1:nsteps,:),  view(δSv,1:nsteps)
end

vareqs_taylorinteg (generic function with 1 method)

In [ ]:
x0

In [4]:
#Initial conditions
c1 = condini(2.57, 0.0)

4-element Array{Float64,1}:
 1.60312
 0.0    
 0.0    
 4.0    

In [11]:
tvS

318-element SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true}:
  0.0      
  0.0949279
  0.189856 
  0.284784 
  0.379712 
  0.47464  
  0.569568 
  0.664495 
  0.759423 
  0.854351 
  0.949279 
  1.04421  
  1.13914  
  ⋮        
 29.0479   
 29.1429   
 29.2378   
 29.3327   
 29.4277   
 29.5226   
 29.6175   
 29.7124   
 29.8074   
 29.9023   
 29.9972   
 30.0      

In [8]:
@time t1, x1 = taylorinteg(undrivenHelium!, c1, 0.0, 30.0, 25, 1e-20, maxsteps=30000 );  

  5.164946 seconds (57.32 M allocations: 5.815 GiB, 17.73% gc time)


In [9]:
x1

30001×4 SubArray{Float64,2,Array{Float64,2},Tuple{UnitRange{Int64},Base.Slice{Base.OneTo{Int64}}},false}:
   1.60312   0.0         0.0         4.0      
   1.60312   0.276405   -0.0033146   3.95213  
   1.60308   0.496706   -0.0176984   3.83571  
   1.60289   0.678943   -0.0388874   3.66518  
   1.60246   0.824189   -0.0539517   3.45254  
   1.60186   0.946972   -0.0483036   3.18103  
   1.60149   1.03795    -0.0102858   2.88229  
   1.60178   1.10858     0.0668074   2.54466  
   1.60321   1.16312     0.18684     2.16577  
   1.60621   1.20414     0.348527    1.74891  
   1.61115   1.23372     0.545863    1.30003  
   1.61838   1.25347     0.770835    0.823581 
   1.62868   1.26429     1.02137     0.308276 
   ⋮                                          
 100.18     -0.0967868  77.8763     -3.98991  
 100.18     -0.974705   77.8769     -2.79754  
 100.183    -1.36364    77.8793     -0.0414106
 100.187    -1.00537    77.8817      2.70256  
 100.187    -0.133927   77.8823      3.98066  
 

In [5]:
@time tvS, xvS, δSv = vareqs_taylorinteg(undrivenHelium!, c1, 0.0, 30.0, 25, 1e-20, maxsteps=30000 );  

  5.764419 seconds (47.55 M allocations: 3.338 GiB, 16.81% gc time)


In [6]:
norm(δSv,Inf)

1.1324508642222405e-16

In [7]:
xvS

318×4 SubArray{Float64,2,Array{Float64,2},Tuple{UnitRange{Int64},Base.Slice{Base.OneTo{Int64}}},false}:
 1.60312  0.0          0.0         4.0    
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 ⋮                                        
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3.96315
 1.60312  0.24322     -0.00227463  3

In [8]:
#Poincare functions
function poincare{T<:Number}(f!, g, q0::Array{T,1}, t0::T, tmax::T,
        order::Int, abstol::T; maxsteps::Int=500, nriter::Int=5)

    # Allocation
    const tv = Array{T}(maxsteps+1)
    dof = length(q0)
    const xv = Array{T}(dof, maxsteps+1)
    const vT = zeros(T, order+1)
    vT[2] = one(T)

    # Initialize the vector of Taylor1 expansions
    const x = Array{Taylor1{T}}(dof)
    const dx = Array{Taylor1{T}}(dof)
    const xaux = Array{Taylor1{T}}(dof)
    for i in eachindex(q0)
        @inbounds x[i] = Taylor1( q0[i], order )
    end

    # Initial conditions
    @inbounds tv[1] = t0
    @inbounds xv[:,1] .= q0
    x0 = copy(q0)

    #Some auxiliary arrays for root-finding/event detection/Poincaré surface of section evaluation
    const g_val = Taylor1(zero(T), order)
    const g_val_old = Taylor1(zero(T), order)
    const slope = zero(T)
    const dt_li = zero(T)
    const dt_nr = zero(T)
    const δt = zero(T)
    const δt_old = zero(T)

    const x_g_Dg_D2g = vcat(x, dx, zero(x[1]), zero(x[1]))
    const x_g_Dg_D2g_val = Array{T}( length(x_g_Dg_D2g) )

    const tvS = similar(tv)
    const xvS = similar(xv)
    const gvS = similar(tv)

    #auxiliary range object for Newton-Raphson iterations
    const nrinds = 1:nriter

    # Integration
    nsteps = 1
    nevents = 1 #number of detected events
    while t0 < tmax
        δt_old = δt
        δt = TaylorIntegration.taylorstep!(f!, x, dx, xaux, t0, tmax, x0, order, abstol, vT)
        g_val = g(t0, x, dx)
        if g_val_old[1]*g_val[1] < zero(T)

            #first guess: linear interpolation
            slope = (g_val[1]-g_val_old[1])/δt_old
            dt_li = -(g_val[1]/slope)

            x_g_Dg_D2g[1:dof] = x
            x_g_Dg_D2g[dof+1:2dof] = dx
            x_g_Dg_D2g[2dof+1] = g_val
            x_g_Dg_D2g[2dof+2] = derivative(g_val)

            #Newton-Raphson iterations
            dt_nr = dt_li
            evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))

            for i in eachindex(nrinds)
                dt_nr = dt_nr-x_g_Dg_D2g_val[2dof+1]/x_g_Dg_D2g_val[2dof+2]
                evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))
            end
            evaluate!(x_g_Dg_D2g[1:2dof], dt_nr, view(x_g_Dg_D2g_val,1:2dof))

            tvS[nevents] = t0+dt_nr
            xvS[:,nevents] .= view(x_g_Dg_D2g_val,1:dof)
            gvS[nevents] = x_g_Dg_D2g_val[2dof+1]

            nevents += 1
        end
        g_val_old = deepcopy(g_val)
        for i in eachindex(x0)
            @inbounds x[i][1] = x0[i]
        end
        t0 += δt
        nsteps += 1
        @inbounds tv[nsteps] = t0
        @inbounds xv[:,nsteps] .= x0
        if nsteps > maxsteps
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
    end

    return view(tv,1:nsteps), view(transpose(xv),1:nsteps,:), view(tvS,1:nevents-1), view(transpose(xvS),1:nevents-1,:), view(gvS,1:nevents-1)
end

#poincare + jet transport
function poincare{T<:Number}(f!, g, q0::Array{TaylorN{T},1}, t0::T, tmax::T,
        order::Int, abstol::T; maxsteps::Int=500, nriter::Int=5)

    # Allocation
    const tv = Array{T}(maxsteps+1)
    dof = length(q0)
    const xv = Array{TaylorN{T}}(dof, maxsteps+1)
    const vT = zeros(T, order+1)
    vT[2] = one(T)

    # Initialize the vector of Taylor1 expansions
    const x = Array{Taylor1{TaylorN{T}}}(dof)
    const dx = Array{Taylor1{TaylorN{T}}}(dof)
    const xaux = Array{Taylor1{TaylorN{T}}}(dof)
    for i in eachindex(q0)
        @inbounds x[i] = Taylor1( q0[i], order )
    end

    # Initial conditions
    @inbounds tv[1] = t0
    @inbounds xv[:,1] .= q0
    x0 = copy(q0)

    #Some auxiliary arrays for root-finding/event detection/Poincaré surface of section evaluation
    zeroTN = zero(q0[1])
    const g_val = Taylor1(zeroTN, order)
    const g_val_old = Taylor1(zeroTN, order)
    const slope = zeroTN
    const dt_li = zeroTN
    const dt_nr = zeroTN
    const δt = zeroTN
    const δt_old = zeroTN

    const x_g_Dg_D2g = vcat(x, dx, zero(x[1]), zero(x[1]))
    const x_g_Dg_D2g_val = Array{TaylorN{T}}( length(x_g_Dg_D2g) )

    const tvS = Array{TaylorN{T}}( length(tv) )
    const xvS = similar(xv)
    const gvS = similar(tvS)

    #auxiliary range object for Newton-Raphson iterations
    const nrinds = 1:nriter

    # Integration
    nsteps = 1
    nevents = 1 #number of detected events
    while t0 < tmax
        δt_old = δt
        δt = TaylorIntegration.taylorstep!(f!, x, dx, xaux, t0, tmax, x0, order, abstol, vT)
        g_val = g(t0, x, dx)
        if g_val_old[1][1][1]*g_val[1][1][1] < zero(T)

            #first guess: linear interpolation
            slope = (g_val[1]-g_val_old[1])/δt_old
            dt_li = -(g_val[1]/slope)

            x_g_Dg_D2g[1:dof] = x
            x_g_Dg_D2g[dof+1:2dof] = dx
            x_g_Dg_D2g[2dof+1] = g_val
            x_g_Dg_D2g[2dof+2] = derivative(g_val)

            #Newton-Raphson iterations
            dt_nr = dt_li
            evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))

            for i in eachindex(nrinds)
                dt_nr = dt_nr-x_g_Dg_D2g_val[2dof+1]/x_g_Dg_D2g_val[2dof+2]
                evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))
            end
            evaluate!(x_g_Dg_D2g[1:2dof], dt_nr, view(x_g_Dg_D2g_val,1:2dof))

            tvS[nevents] = t0+dt_nr
            xvS[:,nevents] .= view(x_g_Dg_D2g_val,1:dof)
            gvS[nevents] = x_g_Dg_D2g_val[2dof+1]

            nevents += 1
        end
        g_val_old = deepcopy(g_val)
        for i in eachindex(x0)
            @inbounds x[i][1] = x0[i]
        end
        t0 += δt
        nsteps += 1
        @inbounds tv[nsteps] = t0
        @inbounds xv[:,nsteps] .= x0
        if nsteps > maxsteps
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
    end

    return view(tv,1:nsteps), transpose(view(xv,:,1:nsteps)), view(tvS,1:nevents-1), transpose(view(xvS,:,1:nevents-1)), view(gvS,1:nevents-1)
end

function poincare2{T<:Number}(f!, g, q0::Array{T,1}, t0::T, tmax::T,
        order::Int, abstol::T; maxsteps::Int=500, nriter::Int=5)

    # Allocation
    const tv = Array{T}(maxsteps+1)
    dof = length(q0)
    const xv = Array{T}(dof, maxsteps+1)
    const vT = zeros(T, order+1)
    vT[2] = one(T)

    # Initialize the vector of Taylor1 expansions
    const x = Array{Taylor1{T}}(dof)
    const dx = Array{Taylor1{T}}(dof)
    const xaux = Array{Taylor1{T}}(dof)
    for i in eachindex(q0)
        @inbounds x[i] = Taylor1( q0[i], order )
    end

    # Initial conditions
    @inbounds tv[1] = t0
    @inbounds xv[:,1] .= q0
    x0 = copy(q0)

    #Some auxiliary arrays for root-finding/event detection/Poincaré surface of section evaluation
    const g_val = Taylor1(zero(T), order)
    const g_val_old = Taylor1(zero(T), order)
    const slope = zero(T)
    const dt_li = zero(T)
    const dt_nr = zero(T)
    const δt = zero(T)
    const δt_old = zero(T)

    const x_g_Dg_D2g = vcat(x, dx, zero(x[1]), zero(x[1]))
    const x_g_Dg_D2g_val = Array{T}( length(x_g_Dg_D2g) )

    const tvS = similar(tv)
    const xvS = similar(xv)
    const gvS = similar(tv)

    #auxiliary range object for Newton-Raphson iterations
    const nrinds = 1:nriter

    # Integration
    nsteps = 1
    nevents = 1 #number of detected events
    while t0 < tmax
        δt_old = δt
        δt = TaylorIntegration.taylorstep!(f!, x, dx, xaux, t0, tmax, x0, order, abstol, vT)
        g_val = g(t0, x, dx)
        if g_val_old[2]*g_val[2] < zero(T)

            #first guess: linear interpolation
            slope = (g_val[2]-g_val_old[2])/δt_old
            dt_li = -(g_val[2]/slope)

            x_g_Dg_D2g[1:dof] = x
            x_g_Dg_D2g[dof+1:2dof] = dx
            x_g_Dg_D2g[2dof+1] = derivative(g_val)
            x_g_Dg_D2g[2dof+2] = derivative(x_g_Dg_D2g[2dof+1])

            #Newton-Raphson iterations
            dt_nr = dt_li
            evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))

            for i in eachindex(nrinds)
                dt_nr = dt_nr-x_g_Dg_D2g_val[2dof+1]/x_g_Dg_D2g_val[2dof+2]
                evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))
            end
            evaluate!(x_g_Dg_D2g[1:2dof], dt_nr, view(x_g_Dg_D2g_val,1:2dof))

            tvS[nevents] = t0+dt_nr
            xvS[:,nevents] .= view(x_g_Dg_D2g_val,1:dof)
            gvS[nevents] = x_g_Dg_D2g_val[2dof+1]

            nevents += 1
        end
        g_val_old = deepcopy(g_val)
        for i in eachindex(x0)
            @inbounds x[i][1] = x0[i]
        end
        t0 += δt
        nsteps += 1
        @inbounds tv[nsteps] = t0
        @inbounds xv[:,nsteps] .= x0
        if nsteps > maxsteps
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
    end

    return view(tv,1:nsteps), view(transpose(xv),1:nsteps,:), view(tvS,1:nevents-1), view(transpose(xvS),1:nevents-1,:), view(gvS,1:nevents-1)
end

#poincare2 + jet transport
function poincare2{T<:Number}(f!, g, q0::Array{TaylorN{T},1}, t0::T, tmax::T,
        order::Int, abstol::T; maxsteps::Int=500, nriter::Int=5)

    # Allocation
    const tv = Array{T}(maxsteps+1)
    dof = length(q0)
    const xv = Array{TaylorN{T}}(dof, maxsteps+1)
    const vT = zeros(T, order+1)
    vT[2] = one(T)

    # Initialize the vector of Taylor1 expansions
    const x = Array{Taylor1{TaylorN{T}}}(dof)
    const dx = Array{Taylor1{TaylorN{T}}}(dof)
    const xaux = Array{Taylor1{TaylorN{T}}}(dof)
    for i in eachindex(q0)
        @inbounds x[i] = Taylor1( q0[i], order )
    end

    # Initial conditions
    @inbounds tv[1] = t0
    @inbounds xv[:,1] .= q0
    x0 = copy(q0)

    #Some auxiliary arrays for root-finding/event detection/Poincaré surface of section evaluation
    zeroTN = zero(q0[1])
    const g_val = Taylor1(zeroTN, order)
    const g_val_old = Taylor1(zeroTN, order)
    const slope = zeroTN
    const dt_li = zeroTN
    const dt_nr = zeroTN
    const δt = zeroTN
    const δt_old = zeroTN

    const x_g_Dg_D2g = vcat(x, dx, zero(x[1]), zero(x[1]))
    const x_g_Dg_D2g_val = Array{TaylorN{T}}( length(x_g_Dg_D2g) )

    const tvS = Array{TaylorN{T}}( length(tv) )
    const xvS = similar(xv)
    const gvS = similar(tvS)

    #auxiliary range object for Newton-Raphson iterations
    const nrinds = 1:nriter

    # Integration
    nsteps = 1
    nevents = 1 #number of detected events
    while t0 < tmax
        δt_old = δt
        δt = TaylorIntegration.taylorstep!(f!, x, dx, xaux, t0, tmax, x0, order, abstol, vT)
        g_val = g(t0, x, dx)
        if g_val_old[2][1][1]*g_val[2][1][1] < zero(T)

            #first guess: linear interpolation
            slope = (g_val[2]-g_val_old[2])/δt_old
            dt_li = -(g_val[2]/slope)

            x_g_Dg_D2g[1:dof] = x
            x_g_Dg_D2g[dof+1:2dof] = dx
            x_g_Dg_D2g[2dof+1] = derivative(g_val)
            x_g_Dg_D2g[2dof+2] = derivative(x_g_Dg_D2g[2dof+1])

            #Newton-Raphson iterations
            dt_nr = dt_li
            evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))

            for i in eachindex(nrinds)
                dt_nr = dt_nr-x_g_Dg_D2g_val[2dof+1]/x_g_Dg_D2g_val[2dof+2]
                evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))
            end
            evaluate!(x_g_Dg_D2g[1:2dof], dt_nr, view(x_g_Dg_D2g_val,1:2dof))

            tvS[nevents] = t0+dt_nr
            xvS[:,nevents] .= view(x_g_Dg_D2g_val,1:dof)
            gvS[nevents] = x_g_Dg_D2g_val[2dof+1]

            nevents += 1
        end
        g_val_old = deepcopy(g_val)
        for i in eachindex(x0)
            @inbounds x[i][1] = x0[i]
        end
        t0 += δt
        nsteps += 1
        @inbounds tv[nsteps] = t0
        @inbounds xv[:,nsteps] .= x0
        if nsteps > maxsteps
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
    end

    return view(tv,1:nsteps), view(transpose(xv),1:nsteps,:), view(tvS,1:nevents-1), view(transpose(xvS),1:nevents-1,:), view(gvS,1:nevents-1)
end

poincare2 (generic function with 2 methods)

In [ ]:
x1[end,:]

In [ ]:
jacobian(x1[end,:])

In [ ]:
length(x1)

In [ ]:
#Ahora obtenemos los jacobianos y las posiciones a partir de los resultados de la integración
function separa{T<:Number}(qs::Array{TaylorN{T},1})
    #Allocations
    ll = length(qs)
    xs = Array{T}(ll)
    js = Array{Array{T,2}}(ll)
    
    #
    

In [ ]:
x1[end,:][1][2]

In [ ]:
typeof(jacobian(x1[end,:]))